In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#infile = "../NTuples/test_truth.root"
#infile = "../NTuples/small_tpceta_sample.root"
#infile = "../NTuples/full_tpceta_sample.root"
infile = "../NTuples/production_sample_take1.root"

In [ ]:

inFile = uproot.open(infile)

#h_tot_pot = ROOT.TFile.Open(infile).Get("TotalPOT")
#pot = h_tot_pot.GetBinContent(1)

slc_tree = inFile["slc_tree"]
particle_tree1 = inFile["particle_tree1"]
particle_tree2 = inFile["particle_tree2"]
daughter_tree1 = inFile["daughter_tree1"]
daughter_tree2 = inFile["daughter_tree2"]

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
particle_df1 = particle_tree1.arrays(particle_tree1.keys(), library="pd")
particle_df2 = particle_tree2.arrays(particle_tree2.keys(), library="pd")
daughter_df1 = daughter_tree1.arrays(daughter_tree1.keys(), library="pd")
daughter_df2 = daughter_tree2.arrays(daughter_tree2.keys(), library="pd")

slc_df[:2]

In [ ]:
particle_df1[:2]

In [ ]:
particle_df2[:2]

In [ ]:
daughter_df1[:2]

In [ ]:
daughter_df2[:2]

In [ ]:
def is_inTPC(row):
    x, y, z = row["vtx_x"], row["vtx_y"], row["vtx_z"]
    if math.isnan(x) or math.isnan(y) or math.isnan(z):
        return 0
    #if (x == 'NaN') or (y == 'NaN') or (z == 'NaN'):
    #    return 0
        
    if (-200 < x < 200) and (-200 < y < 200) and (0 < z < 500):
        return 1
    else:
        return 0

slc_df["inTPC"] = slc_df.apply(is_inTPC, axis=1)
slc_df[:2]

In [ ]:
topology_labels = {
    0:r"$\nu_{\mu} CC$",
    1:r"$\nu_{\mu} NC$",
    2:r"$\nu_{e} CC$",
    3:r"$\nu_{e} NC$",
    4:r"$\bar{\nu}_{\mu}$",
    5:r"$\bar{\nu}_{e}$",
    6:r"DIRT $\nu$",
    7:"Cosmic",
}

topology_selections = {
    0:"pdg == 14.0 and iscc == 1.0 and inTPC == 1",
    1:"pdg == 14.0 and isnc == 1.0 and inTPC == 1",
    2:"pdg == 12.0 and iscc == 1.0 and inTPC == 1",
    3:"pdg == 12.0 and isnc == 1.0 and inTPC == 1",
    4:"pdg == -14.0 and inTPC == 1",
    5:"pdg == -12.0 and inTPC == 1",
    6:"(pdg == 14.0 or pdg == 12.0 or pdg == -14.0 or pdg == -12.0 ) and inTPC == 0",
    7:"pdg == -1",
}

In [ ]:
# Let's look at the file composition

N = slc_df.shape[0]

bin_centers = [1]
bin_width = 1
N_prev = 0
#plt.yscale('log')
for num in range(len(topology_labels.keys())):
    temp_df = slc_df.query(topology_selections[num])
    N_top = temp_df.shape[0]
    
    plt.bar(bin_centers, [N_top], width=bin_width, bottom=N_prev, alpha=0.6, label=topology_labels[num])
    N_prev += N_top

plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.xticks([])
plt.ylabel("Slice Count", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
# Let's add the topologies to the dataframes

# initialize to -1 for now
slc_df["TOP"] = -1

for num in range(len(topology_selections.keys())):
    #temp = slc_df.query(topology_selections[num])
    condition = slc_df.index.isin(slc_df.query(topology_selections[num]).index)
    slc_df.loc[condition, "TOP"] = num
    
slc_df[:2]


In [ ]:


#df_result = df_large.merge(df_small, on=['col1', 'col2'], how='left')

# Let's try to add the topology to the particle dfs
df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'TOP']]
particle_df1 = particle_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

particle_df1[:2]

In [ ]:
daughter_df1 = daughter_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

daughter_df1[:2]

In [ ]:
particle_df2["TOP"] = particle_df1["TOP"]
daughter_df2["TOP"] = daughter_df1["TOP"]

particle_df2[:2]

In [ ]:

cosmic_primaries = particle_df2.query("TOP == 7")
cosmic_primaries[:2]

#plt.hist(particle_df2.query("TOP == 7")["start_x"].values, bins=100, label=topology_labels[7])
#plt.legend()
#plt.show()

In [ ]:
def is_prim_cont(row):
    run, subrun, evt, slc = row["run"], row["subrun"], row["evt"], row["slc"]
    q = "run == "+str(run)+" and subrun == "+str(subrun) + " and evt == "+str(evt) + "and slc == "+str(slc)
    temp_df = particle_df2.query(q)
    x_s, y_s, z_s = temp_df["start_x"].values, temp_df["start_y"].values, temp_df["start_z"].values
    x_e, y_e, z_e = temp_df["end_x"].values, temp_df["end_y"].values, temp_df["end_z"].values
    for num in range(len(x_s)):
        if (x_s[num] > 200 or x_s[num] < -200):
            return 0
        if (x_e[num] > 200 or x_e[num] < -200):
            return 0
        if (y_s[num] > 200 or y_s[num] < -200):
            return 0
        if (y_e[num] > 200 or y_e[num] < -200):
            return 0
        if (z_s[num] > 500 or z_s[num] < 0):
            return 0
        if (z_e[num] > 500 or z_e[num] < 0):
            return 0
        return 1


  
slc_df["primCont"] = slc_df.apply(is_prim_cont, axis=1)    

slc_df[:2]

In [ ]:
# Let's look at the file composition


bin_centers = [0, 1]
bin_width = 1
N_prev_sel = 0
N_prev_rej = 0
for num in range(len(topology_labels.keys())):
    temp_df = slc_df.query(topology_selections[num])
    N_top = temp_df.shape[0]
    N_sel = temp_df.query("primCont == 1").shape[0]
    N_rej = N_top - N_sel
    
    plt.bar(bin_centers, [N_sel, N_rej], width=bin_width, bottom=[N_prev_sel, N_prev_rej], 
            alpha=0.6, label=topology_labels[num])
    N_prev_sel += N_sel
    N_prev_rej += N_rej

#plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.legend()
#plt.xticks([])
plt.xticks([0, 1], ['Contained in TPC', 'Not Contained'])
plt.ylabel("Slice Count", fontsize=14)
plt.ylim([0, N])
plt.title("All Primary Particles")
plt.show()